## Disreperancy Global IPC aggregation methods
For some dates different results are gained when calculating the numbers on admin0 and admin1 level in different way. Since the numbers are directly reported in the excel sheet on admin0 level we can use those. Simeltaneously we could sum all the numbers reported on admin2 level and this should give the same results. However, they don't match. This notebook does a short exploration of where the differences occur

In [56]:
import pandas as pd

In [57]:
admin_level=0
country="malawi"

In [58]:
df=pd.read_excel(f"../Data/GlobalIPC/{country}_globalipc_newcolumnnames.xlsx",index_col=0)
df.loc[:,"date"] = pd.to_datetime(df.loc[:,"date"])

In [60]:
df = df[(df["date"].notnull()) & (df[f"ADMIN{admin_level}"].notnull())]

In [61]:
df.head()

,ADMIN0,ADMIN1,ADMIN2,ADMIN2_ID,Analysis Name,date,Country Population,pop_CS,% of total county Pop,Area Phase,...,ML2_2,perc_ML2_2,ML2_3,perc_ML2_3,ML2_4,perc_ML2_4,ML2_5,perc_ML2_5,ML2_3p,perc_ML2_3p
0,Malawi: Acute Food Insecurity Analysis Novembe...,NaN,NaN,NaN,NaN,2020-12-01,19718415.0,17677952.0,0.89652,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Malawi,NaN,Balaka,21180610.0,Acute Food Insecurity Analysis November 2020,2020-12-01,NaN,389024.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Malawi,NaN,Blantyre,21180584.0,Acute Food Insecurity Analysis November 2020,2020-12-01,NaN,451220.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Malawi,NaN,Blantyre City,21180722.0,Acute Food Insecurity Analysis November 2020,2020-12-01,NaN,1072684.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Malawi,NaN,Chikhwawa,21180618.0,Acute Food Insecurity Analysis November 2020,2020-12-01,NaN,551538.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_agg = df[df["ADMIN0"].str.lower().str.fullmatch(country.lower())].groupby(["ADMIN0","date"],as_index=False).sum()
df_precalc = df[df["ADMIN0"].str.lower().str.match(f"{country.lower()}:")]
# df_adm1agg = df[~df["ADMIN0"].str.lower().str.contains(f"{country.lower()}")].groupby("date",as_index=False).sum()

In [63]:
df_agg["pop_CS_sum"]=df_agg[[f"CS_{i}" for i in range(1,6)]].sum(axis=1)
df_precalc["pop_CS_sum"]=df_precalc[[f"CS_{i}" for i in range(1,6)]].sum(axis=1)
df_agg["pop_CS_diff"]=df_agg["pop_CS"]-df_agg["pop_CS_sum"]
df_precalc["pop_CS_diff"]=df_precalc["pop_CS"]-df_precalc["pop_CS_sum"]

<ipython-input-63-f4f01cf85269>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precalc["pop_CS_sum"]=df_precalc[[f"CS_{i}" for i in range(1,6)]].sum(axis=1)
<ipython-input-63-f4f01cf85269>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precalc["pop_CS_diff"]=df_precalc["pop_CS"]-df_precalc["pop_CS_sum"]


In [64]:
df_agg[["date","pop_CS","pop_CS_sum","pop_CS_diff"]]

,date,pop_CS,pop_CS_sum,pop_CS_diff
0,2017-07-01,14110120.0,13775745.0,334375.0
1,2018-08-01,15329133.0,15367177.0,-38044.0
2,2019-06-01,14735053.0,14727588.0,7465.0
3,2019-11-01,0.0,0.0,0.0
4,2020-08-01,17678952.0,17678955.0,-3.0
5,2020-12-01,17677952.0,17677958.0,-6.0


In [65]:
df_precalc[["date","pop_CS","pop_CS_sum","pop_CS_diff"]].sort_values("date")

,date,pop_CS,pop_CS_sum,pop_CS_diff
155,2017-07-01,14110120.0,14065000.0,45120.0
126,2018-08-01,15329133.0,15241027.0,88106.0
97,2019-06-01,14734053.0,14734053.0,0.0
68,2019-11-01,NaN,0.0,NaN
34,2020-08-01,17678952.0,17678952.0,0.0
0,2020-12-01,17677952.0,17677958.0,-6.0


In [66]:
df_comb=df_agg.merge(df_precalc,on="date",suffixes=("_agg","_prec"))#.merge(df_adm1agg.rename(columns={"pop_CS":"pop_CS_adm1"}),on='date')

In [67]:
df_comb

,ADMIN0_agg,date,ADMIN1_agg,ADMIN2_ID_agg,Country Population_agg,pop_CS_agg,% of total county Pop_agg,Area Phase_agg,CS_1_agg,perc_CS_1_agg,...,ML2_3_prec,perc_ML2_3_prec,ML2_4_prec,perc_ML2_4_prec,ML2_5_prec,perc_ML2_5_prec,ML2_3p_prec,perc_ML2_3p_prec,pop_CS_sum_prec,pop_CS_diff_prec
0,Malawi,2017-07-01,0.0,328117314.0,0.0,14110120.0,0.0,37.0,11520844.0,2157.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14065000.0,45120.0
1,Malawi,2018-08-01,0.0,346310381.0,0.0,15329133.0,0.0,58.0,8625171.0,1492.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15241027.0,88106.0
2,Malawi,2019-06-01,0.0,367385706.0,0.0,14735053.0,0.0,39.0,11138502.0,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14734053.0,0.0
3,Malawi,2019-11-01,0.0,379151766.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,Malawi,2020-08-01,0.0,593401312.0,0.0,17678952.0,0.0,62.0,10457215.0,1945.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17678952.0,0.0
5,Malawi,2020-12-01,0.0,677779536.0,0.0,17677952.0,0.0,70.0,8945225.0,1665.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17677958.0,-6.0


In [71]:
df_comb["pop_CS_diff_aggprec"]=df_comb["pop_CS_agg"]-df_comb["pop_CS_prec"]
df_comb["CS_1_diff_aggprec"]=df_comb["CS_1_agg"]-df_comb["CS_1_prec"]
df_comb["CS_2_diff_aggprec"]=df_comb["CS_2_agg"]-df_comb["CS_2_prec"]
df_comb["CS_3_diff_aggprec"]=df_comb["CS_3_agg"]-df_comb["CS_3_prec"]
df_comb["CS_4_diff_aggprec"]=df_comb["CS_4_agg"]-df_comb["CS_4_prec"]
df_comb["CS_5_diff_aggprec"]=df_comb["CS_5_agg"]-df_comb["CS_5_prec"]
# df_comb["pop_CS_diff_aggadm1"]=df_comb["pop_CS_agg"]-df_comb["pop_CS_adm1"]
# df_comb["pop_CS_diff_precadm1"]=df_comb["pop_CS_prec"]-df_comb["pop_CS_adm1"]

In [72]:
df_comb[["date","pop_CS_agg","pop_CS_prec","pop_CS_diff_aggprec"]] #"pop_CS_adm1","pop_CS_diff_aggadm1","pop_CS_diff_precadm1"]]

,date,pop_CS_agg,pop_CS_prec,pop_CS_diff_aggprec
0,2017-07-01,14110120.0,14110120.0,0.0
1,2018-08-01,15329133.0,15329133.0,0.0
2,2019-06-01,14735053.0,14734053.0,1000.0
3,2019-11-01,0.0,NaN,NaN
4,2020-08-01,17678952.0,17678952.0,0.0
5,2020-12-01,17677952.0,17677952.0,0.0


In [73]:
df_comb[["date","pop_CS_agg","pop_CS_prec","pop_CS_sum_agg","pop_CS_sum_prec"]] #"pop_CS_adm1","pop_CS_diff_aggadm1","pop_CS_diff_precadm1"]]

,date,pop_CS_agg,pop_CS_prec,pop_CS_sum_agg,pop_CS_sum_prec
0,2017-07-01,14110120.0,14110120.0,13775745.0,14065000.0
1,2018-08-01,15329133.0,15329133.0,15367177.0,15241027.0
2,2019-06-01,14735053.0,14734053.0,14727588.0,14734053.0
3,2019-11-01,0.0,NaN,0.0,0.0
4,2020-08-01,17678952.0,17678952.0,17678955.0,17678952.0
5,2020-12-01,17677952.0,17677952.0,17677958.0,17677958.0


In [74]:
df_comb[["date","CS_1_diff_aggprec","CS_2_diff_aggprec","CS_3_diff_aggprec","CS_4_diff_aggprec","CS_5_diff_aggprec"]]

,date,CS_1_diff_aggprec,CS_2_diff_aggprec,CS_3_diff_aggprec,CS_4_diff_aggprec,CS_5_diff_aggprec
0,2017-07-01,-114156.0,-118851.0,-56248.0,0.0,0.0
1,2018-08-01,2.0,0.0,6778.0,119370.0,0.0
2,2019-06-01,-6885.0,391.0,29.0,0.0,0.0
3,2019-11-01,NaN,NaN,NaN,NaN,NaN
4,2020-08-01,2.0,2.0,-1.0,0.0,0.0
5,2020-12-01,0.0,0.0,0.0,0.0,0.0


In [75]:
df_comb[["date","CS_2_agg","CS_2_prec"]] 

,date,CS_2_agg,CS_2_prec
0,2017-07-01,1890149.0,2009000.0
1,2018-08-01,4480354.0,4480354.0
2,2019-06-01,2915890.0,2915499.0
3,2019-11-01,0.0,NaN
4,2020-08-01,5524860.0,5524858.0
5,2020-12-01,6183030.0,6183030.0


In [76]:
df_comb[["date","CS_3_agg","CS_3_prec"]] 

,date,CS_3_agg,CS_3_prec
0,2017-07-01,364752.0,421000.0
1,2018-08-01,1819656.0,1812878.0
2,2019-06-01,673196.0,673167.0
3,2019-11-01,0.0,NaN
4,2020-08-01,1696880.0,1696881.0
5,2020-12-01,2549703.0,2549703.0


In [77]:
df_comb[["date","CS_4_agg","CS_4_prec"]] 

,date,CS_4_agg,CS_4_prec
0,2017-07-01,0.0,0.0
1,2018-08-01,441996.0,322626.0
2,2019-06-01,0.0,0.0
3,2019-11-01,0.0,NaN
4,2020-08-01,0.0,0.0
5,2020-12-01,0.0,0.0


In [78]:
df_comb[["date","CS_5_agg","CS_5_prec"]] 

,date,CS_5_agg,CS_5_prec
0,2017-07-01,0.0,0.0
1,2018-08-01,0.0,0.0
2,2019-06-01,0.0,0.0
3,2019-11-01,0.0,NaN
4,2020-08-01,0.0,0.0
5,2020-12-01,0.0,0.0


### ADMIN1

In [79]:
df_adm1_precalc = df[~df["ADMIN0"].str.lower().str.contains(f"{country.lower()}")]
df_adm1_precalc=df_adm1_precalc.drop("ADMIN1",axis=1)
df_adm1_precalc=df_adm1_precalc.rename(columns={"ADMIN0":"ADMIN1"})

In [80]:
admin_level=2

In [81]:
df_adm2 = df[(df["date"].notnull()) & (df[f"ADMIN{admin_level}"].notnull())]

In [82]:
df_adm1_agg = df_adm2.groupby(["date", "ADMIN1"], dropna=False, as_index=False).sum()

In [83]:
df_adm1_agg.head()

,date,ADMIN1,ADMIN2_ID,Country Population,pop_CS,% of total county Pop,Area Phase,CS_1,perc_CS_1,CS_2,...,ML2_2,perc_ML2_2,ML2_3,perc_ML2_3,ML2_4,perc_ML2_4,ML2_5,perc_ML2_5,ML2_3p,perc_ML2_3p
0,2017-07-01,NaN,328117314.0,0.0,14110120.0,0.0,37.0,11520844.0,2157.0,1890149.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-08-01,NaN,346310381.0,0.0,15329133.0,0.0,58.0,8625171.0,1492.0,4480354.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-06-01,NaN,367385706.0,0.0,14735053.0,0.0,39.0,11138502.0,2018.0,2915890.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-11-01,NaN,379151766.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-08-01,NaN,593401312.0,0.0,17678952.0,0.0,62.0,10457215.0,1945.0,5524860.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
df_adm1_precalc.equals(df_adm1_agg)

False

In [85]:
df_adm1_precalc.shape

(0, 54)

In [86]:
df_adm1_agg.shape

(6, 50)

In [87]:
df_adm1_comb=df_adm1_agg.merge(df_adm1_precalc,on=["date","ADMIN1"],suffixes=("_agg","_prec"))

In [88]:
df_adm1_comb["pop_CS_diff"]=df_adm1_comb["pop_CS_agg"]-df_adm1_comb["pop_CS_prec"]

In [89]:
df_adm1_comb[df_adm1_comb["date"]=="2017-07-01"][["ADMIN1","pop_CS_agg","pop_CS_prec","pop_CS_diff"]]

,ADMIN1,pop_CS_agg,pop_CS_prec,pop_CS_diff


In [90]:
df_adm1_comb[df_adm1_comb["date"]=="2020-09-01"][["ADMIN1","pop_CS_agg","pop_CS_prec","pop_CS_diff"]]

,ADMIN1,pop_CS_agg,pop_CS_prec,pop_CS_diff


### ADMIN2

In [91]:
df=pd.read_excel(f"../Data/GlobalIPC/{country}_globalipc_newcolumnnames.xlsx",index_col=0)
df.loc[:,"date"] = pd.to_datetime(df.loc[:,"date"])

In [92]:
df_adm2 = df[(df["date"].notnull()) & (df[f"ADMIN{admin_level}"].notnull())]

In [93]:
df_adm2["pop_CS_sum"]=df_adm2[[f"CS_{i}" for i in range(1,6)]].sum(axis=1)
df_adm2["pop_diff"]=df_adm2["pop_CS"]-df_adm2["pop_CS_sum"]

<ipython-input-93-4dba5bd71eae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adm2["pop_CS_sum"]=df_adm2[[f"CS_{i}" for i in range(1,6)]].sum(axis=1)
<ipython-input-93-4dba5bd71eae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adm2["pop_diff"]=df_adm2["pop_CS"]-df_adm2["pop_CS_sum"]


In [97]:
df_adm2[["date","ADMIN2","pop_CS","pop_CS_sum","pop_diff"]].dropna().sort_values(by="pop_diff")

,date,ADMIN2,pop_CS,pop_CS_sum,pop_diff
142,2018-08-01,Mzimba,980374.0,990178.0,-9804.0
135,2018-08-01,Kasungu,927543.0,936818.0,-9275.0
153,2018-08-01,Zomba,698636.0,705622.0,-6986.0
129,2018-08-01,Chikhwawa,601250.0,607264.0,-6014.0
134,2018-08-01,Karonga,370370.0,374074.0,-3704.0
...,...,...,...,...,...
166,2017-07-01,Machinga,612759.0,594376.0,18383.0
182,2017-07-01,Zomba,655534.0,635868.0,19666.0
171,2017-07-01,Mzimba,921621.0,893973.0,27648.0
165,2017-07-01,Lilongwe,1510579.0,1480367.0,30212.0
